## test netcdf+

In [1]:
import openpathsampling as paths
from openpathsampling.storage.netcdfplus import NetCDFPlus
import numpy as np

### open file

try to create a new storage

In [2]:
st = NetCDFPlus('test_netcdfplus.nc', mode='w')

In [3]:
st.add('details', paths.storage.ObjectStore(paths.Details, has_name=False))

In [4]:
st.details._init()
st.details.set_caching(10)

### create variables types

In [11]:
print sorted(st.get_var_types())

['bool', 'float', 'index', 'int', 'json', 'lazyobj.details', 'length', 'long', 'numpy.float32', 'numpy.float64', 'numpy.int16', 'numpy.int32', 'numpy.int64', 'numpy.int8', 'numpy.uint16', 'numpy.uint32', 'numpy.uint64', 'numpy.uint8', 'obj.details', 'store', 'str']


In [12]:
for var_type in st.get_var_types():
    st.create_variable(var_type, var_type, dimensions=(2, 2, 2))

In [13]:
st.update_delegates()

In [14]:
for var in sorted(st.variables):
    print var

bool
details_json
float
index
int
json
lazyobj.details
length
long
numpy.float32
numpy.float64
numpy.int16
numpy.int32
numpy.int64
numpy.int8
numpy.uint16
numpy.uint32
numpy.uint64
numpy.uint8
obj.details
store
str


In [15]:
for var in sorted(st.vars):
    print var

bool
details_json
float
index
int
json
lazyobj.details
length
long
numpy.float32
numpy.float64
numpy.int16
numpy.int32
numpy.int64
numpy.int8
numpy.uint16
numpy.uint32
numpy.uint64
numpy.uint8
obj.details
store
str


Bool

In [16]:
st.vars['bool'][:] = True

In [17]:
print st.vars['bool'][:]

[[[True, True], [True, True]], [[True, True], [True, True]]]


Float

In [18]:
st.vars['float'][1,1] = 1.0

In [19]:
print st.vars['float'][:]

[[[None, None], [None, None]], [[None, None], [1.0, 1.0]]]


Index

In [20]:
st.vars['index'][0,1,0] = 10
st.vars['index'][0,1,1] = -1

In [21]:
print st.vars['index'][0,1]

[10, None]


Int

In [22]:
st.vars['int'][0,1,0] = 10
st.vars['int'][0,1,1] = -1

In [23]:
print st.vars['int'][:]

[[[None, None], [10, -1]], [[None, None], [None, None]]]


JSON

A JSON serializable object. This can be normal very simple python objects, plus numpy arrays, and objects that implement `to_dict` and `from_dict`.

In [24]:
st.vars['json'][0,1,1] = {'Hallo': 2, 'Test': 3}

In [25]:
st.vars['json'][0,1,0]

In [26]:
st.vars['json'][0,1,0] = paths.Details()

In [27]:
print st.variables['json'][0,1,:]

[u'{"_cls": "Details", "_dict": {}}' u'{"Test": 3, "Hallo": 2}']


All object types registered as being Storable by subclassing from `openpathsampling.base.StorableObject`. For this test we only register type `paths.Snapshot`

Numpy

In [28]:
st.vars['numpy.float32'][:] = np.ones((2,2,2)) * 3.0
st.vars['numpy.float32'][0] = np.ones((2,2)) * 7.0

In [29]:
print st.vars['numpy.float32'][:]

[[[ 7.  7.]
  [ 7.  7.]]

 [[ 3.  3.]
  [ 3.  3.]]]


Obj

In [24]:
st.vars['obj.details'][0,0,0] = paths.Details(cool='First')
st.vars['obj.details'][0,0,0] = paths.Details(cool='Second')
st.vars['obj.details'][0,0,0] = paths.Details(cool='Third')

In [25]:
print st.variables['obj.details'][:]
print st.variables['details_json'][:]

[[[2 --]
  [-- --]]

 [[-- --]
  [-- --]]]
[u'{"_cls": "Details", "_dict": {"cool": "First"}}'
 u'{"_cls": "Details", "_dict": {"cool": "Second"}}'
 u'{"_cls": "Details", "_dict": {"cool": "Third"}}']


In [26]:
print st.vars['obj.details'][0,0,0]
print type(st.vars['obj.details'][0,0,0])

cool = Third

<class 'openpathsampling.pathmover.Details'>


lazy

In [27]:
st.vars['lazyobj.details'][0,0,0] = paths.Details(lazy='First')
st.vars['lazyobj.details'][0,0,0] = paths.Details(lazy='Second')
st.vars['lazyobj.details'][0,0,0] = paths.Details(lazy='Third')

In [28]:
proxy = st.vars['lazyobj.details'][0,0,0]
print 'Type:   ', type(proxy)
print 'Class:  ', proxy.__class__
print 'Content:', proxy.__subject__.__dict__
print 'Access: ', proxy.lazy

Type:    <class 'openpathsampling.storage.objproxy.LoaderProxy'>
Class:   <class 'openpathsampling.pathmover.Details'>
Content: {'lazy': 'Third'}
Access:  Third


### load/save objects

Note that there are now 6 objects. 3 from `obj.details` and `lazyobj.details` each.

In [29]:
print st.details[:]

[<openpathsampling.pathmover.Details object at 0x10f0c4bd0>, <openpathsampling.pathmover.Details object at 0x1061e4f10>, <openpathsampling.pathmover.Details object at 0x10f0c4e50>, <openpathsampling.pathmover.Details object at 0x10f0c4e10>, <openpathsampling.pathmover.Details object at 0x10f0c4d50>, <openpathsampling.pathmover.Details object at 0x10f22afd0>]


In [30]:
obj = paths.Details(name='BlaBla')
st.details.save(obj)
st.save(obj);

In [31]:
print len(st.details)

7


Note that we have only 7 objets since both save operations will only store one object.

Get the index of the obj in the storage

In [32]:
print st.details.variables['json'][st.idx(obj)]
print st.details.vars['json'][st.idx(obj)]
print st.details.vars['json'][st.idx(obj)] is obj
print st.details[st.idx(obj)] is obj

{"_cls": "Details", "_dict": {"name": "BlaBla"}}
name = BlaBla

False
True
